## Inference Demo
**Step 1.** Set these variables

In [ ]:
CKPT_PATH = "./lightning_logs/roberta-base/version_0/checkpoints/epoch=3-step=264.ckpt"
ENCODER_NAME = "roberta-base"

**Step 2.** Load the imports

In [ ]:
from models import BERTEncoderForWordClassification, POSClassifier
from transformers import logging

logging.set_verbosity_error()

**Step 3.** Create the model

In [ ]:
bert = BERTEncoderForWordClassification(ENCODER_NAME)
model = POSClassifier.load_from_checkpoint(CKPT_PATH)
model.set_encoder(bert)

**Step 4.** Perform inference!

In [ ]:
model.infer("Hello guys, I am a dog")

**Step 5.** Profit?